In [ ]:
# Face Recognition, Liveness & Mask Detection with Attendance Restrictions
import numpy as np
import cv2
from scipy.spatial import distance as dist
import pickle
from datetime import datetime, timedelta
from deepface import DeepFace
import mysql.connector
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import mediapipe as mp
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import faiss
import time


DATABASE_PATHS = {
    'a': 'embeddings_section_a.pkl',
    'b': 'embeddings_section_b.pkl',
    'c': 'embeddings_section_c.pkl'
}
GOOGLE_CREDENTIALS_FILE = "credentials.json" # Download your service account key and rename it to this.
MASK_MODEL_PATH = "mask-detector-model.model"

SIMILARITY_THRESHOLD = 0.63

BLINK_THRESHOLD = 0.22
CONSECUTIVE_FRAMES = 1
LIVENESS_BLINKS_REQUIRED = 2 
NEW_USER_IMAGES_COUNT = 20 
IMAGE_CAPTURE_DELAY = 1 
ATTENDANCE_INTERVAL_HOURS = 1


MICRO_MOVEMENT_THRESHOLD = 0.005 
CONSECUTIVE_STATIC_FRAMES = 30 

DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "YOUR_SECURE_PASSWORD"  # <<< IMPORTANT: Change this to your actual MySQL root password!
DB_NAME = "attendance_db"


mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils


index = None
id_to_name = []
selected_section = None


try:
    mask_model = load_model(MASK_MODEL_PATH)
    print("Mask detection model loaded successfully.")
except Exception as e:
    print(f"Error loading mask model: {e}")
    mask_model = None

liveness_data = {}
last_known_faces_data = {}
previous_landmarks = {} 

def get_database_path(section):
    """ Returns the correct database file path for a given section. """
    return DATABASE_PATHS.get(section.lower(), None)

def load_face_database(section):
    global index, id_to_name
    db_path = get_database_path(section)
    face_database = []

    if not db_path:
        print(f"Invalid section: {section}")
        return face_database

    try:
        with open(db_path, 'rb') as f:
            face_database = pickle.load(f)
        print(f"Face database for Section {section} loaded successfully.")
    except FileNotFoundError:
        print(f"Face database not found at '{db_path}'. Creating a new, empty database.")
        return []

    for entry in face_database:
        if "embeddings" in entry:  # new format
            entry["embeddings"] = [l2_normalize(np.array(e, dtype=np.float32)) for e in entry["embeddings"]]
        elif "embedding" in entry:  # old format fallback
            emb = np.array(entry["embedding"], dtype=np.float32)
            entry["embeddings"] = [l2_normalize(emb)]
            del entry["embedding"]  # clean up old key
        else:
            print(f" Skipping malformed entry: {entry}")
    # Build FAISS Index
    all_embeddings = []
    id_to_name = []
    for entry in face_database:
        for emb in entry['embeddings']:
            all_embeddings.append(emb.astype('float32'))
            id_to_name.append(entry['name'])

    if all_embeddings:
        d = len(all_embeddings[0])
        index = faiss.IndexFlatIP(d)   # Inner Product
        index.add(np.array(all_embeddings, dtype='float32'))
        print(f"FAISS index built with {index.ntotal} embeddings for Section {section}.")
    else:
        index = None
        print(f"No embeddings found for Section {section}. FAISS index not built.")

    return face_database

def save_face_database(face_database, section):
    """ Saves the face embeddings database to the file for the given section. """
    db_path = get_database_path(section)
    if not db_path:
        print(f"Cannot save to invalid section: {section}")
        return False
    try:
        with open(db_path, 'wb') as f:
            pickle.dump(face_database, f)
        print(f"Successfully saved face database for Section {section}!")
        return True
    except Exception as e:
        print(f"Error saving face database for Section {section}: {e}")
        return False

def eye_aspect_ratio(eye_landmarks):
    """ Calculates the Eye Aspect Ratio (EAR) for a single eye using MediaPipe landmarks. """
    P2 = eye_landmarks[1]
    P3 = eye_landmarks[2]
    P5 = eye_landmarks[4]
    P6 = eye_landmarks[5]
    P1 = eye_landmarks[0]
    P4 = eye_landmarks[3]

    A = dist.euclidean([P2.x, P2.y], [P6.x, P6.y])
    B = dist.euclidean([P3.x, P3.y], [P5.x, P5.y])
    C = dist.euclidean([P1.x, P1.y], [P4.x, P4.y])

    ear = (A + B) / (2.0 * C)
    return ear

def detect_mask(face_img):
    """ Predicts if a person is wearing a mask using the loaded model. """
    if mask_model is None:
        return "N/A", (128, 128, 128) # Gray color
    
    face_resized = cv2.resize(face_img, (224, 224))
    arr = img_to_array(face_resized) / 255.0
    arr = np.expand_dims(arr, axis=0)

    try:
        (mask, withoutMask) = mask_model.predict(arr, verbose=0)[0]
    except Exception as e:
        print(f"Error during mask prediction: {e}")
        return "N/A", (128, 128, 128)
    
    if mask > withoutMask:
        return "Mask dectected. Please remove it.", (0, 255, 0) # Green
    else:
        return "No Mask", (0, 0, 255) # Red

def calculate_micro_movement(current_landmarks, person_name, frame_w, frame_h):
    """
    Analyzes micro-movements of key facial landmarks.
    """
    global previous_landmarks
    
    # Check if we have previous landmarks for this person
    if person_name not in previous_landmarks:
        previous_landmarks[person_name] = current_landmarks
        return 0.0

    prev_lms = previous_landmarks[person_name]
    displacement_sum = 0.0
    
    # A mix of nose, chin, and eye corners
    key_points_indices = [1, 6, 205, 33, 263, 164, 442, 272]
    
    for i in key_points_indices:
        p_curr = current_landmarks[i]
        p_prev = prev_lms[i]
        
        # Calculate Euclidean distance between the current and previous points
        dx = (p_curr.x - p_prev.x) * frame_w
        dy = (p_curr.y - p_prev.y) * frame_h
        displacement = np.sqrt(dx**2 + dy**2)
        displacement_sum += displacement

    # Calculate average displacement across all key points
    avg_displacement = displacement_sum / len(key_points_indices)
    
    # Update previous landmarks for the next frame
    previous_landmarks[person_name] = current_landmarks
    
    return avg_displacement

# ---------------- GOOGLE SHEETS SETUP ----------------
def setup_google_sheets():
    try:
        scope = ["https://spreadsheets.google.com/feeds",
                 "https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_CREDENTIALS_FILE, scope)
        client = gspread.authorize(creds)

        # Open main sheet
        spreadsheet = client.open("Face_Attendance")

        # Map sections to their worksheet tabs
        sheet_dict = {
            'a': spreadsheet.worksheet("Section_A"),
            'b': spreadsheet.worksheet("Section_B"),
            'c': spreadsheet.worksheet("Section_C")
        }
        print(" Connected to Google Sheets (Sections A, B, C).")
        return sheet_dict
    except Exception as e:
        print(f"Google Sheets setup error: {e}")
        return {}

def log_attendance_to_gsheet(sheet_dict, person_name, section):
    try:
        current_time = datetime.now()
        log_date = current_time.strftime("%Y-%m-%d")
        log_time = current_time.strftime("%H:%M:%S")

        # Select correct worksheet by section
        worksheet = sheet_dict.get(section.lower())
        if worksheet:
            
            records = worksheet.get_all_records()
            records = [{k.lower(): v for k, v in row.items()} for row in records]
            last_entry_time = None

            for row in reversed(records):  # iterate backwards (latest entries first)
                if row["name"] == person_name and row["date"] == log_date:
                    last_entry_time = datetime.strptime(
                        f"{row['date']} {row['time']}", "%Y-%m-%d %H:%M:%S"
                    )
                    break

            if last_entry_time:
                if current_time - last_entry_time < timedelta(hours=ATTENDANCE_INTERVAL_HOURS):
                    print(f"Attendance already logged for {person_name} within the last {ATTENDANCE_INTERVAL_HOURS} hour(s).")
                    return False


            worksheet.append_row([person_name, log_date, log_time])
            print(f"Attendance logged for {person_name} in Section {section.upper()} (Google Sheets).")
            return True
        else:
            print(f"No worksheet found for section {section}")
            return False
    except Exception as e:
        print(f"Error logging to Google Sheets: {e}")
        return False
    
# ---------------- MYSQL SETUP ----------------
def setup_mysql_database():
    try:
        conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, passwd=DB_PASSWORD)
        cursor = conn.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
        conn.close()
        conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, passwd=DB_PASSWORD, database=DB_NAME)
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS attendance_a (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            date DATE NOT NULL,
            time TIME NOT NULL
        )
    ''')
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS attendance_b(
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            date DATE NOT NULL,
            time TIME NOT NULL
        )
     ''')
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS attendance_c (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            date DATE NOT NULL,
            time TIME NOT NULL
        )
    ''')
            
        conn.commit()
        conn.close()
        print(f"MySQL Database '{DB_NAME}' setup successful.")
    except mysql.connector.Error as err:
        print(f" MySQL Error: {err}")



def log_attendance_to_mysql(person_name, section):
    try:
        conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, passwd=DB_PASSWORD, database=DB_NAME)
        cursor = conn.cursor()

        table_name = f"attendance_{section.lower()}"
        log_date = datetime.now().date()
        log_time = datetime.now().time()

        cursor.execute(
            f"SELECT time FROM {table_name} WHERE name=%s AND date=%s ORDER BY time DESC LIMIT 1",
            (person_name, log_date)
        )
        last_entry = cursor.fetchone()

        if last_entry:
            last_time_value = last_entry[0]
            if isinstance(last_time_value, str):
                try:
                    # Try parsing with microseconds
                    last_time_value = datetime.strptime(last_time_value, "%H:%M:%S.%f").time()
                except ValueError:
                    # Fallback: parse without microseconds
                    last_time_value = datetime.strptime(last_time_value, "%H:%M:%S").time()

            last_time = datetime.combine(log_date, last_time_value)
            if datetime.now() - last_time < timedelta(hours=ATTENDANCE_INTERVAL_HOURS):
                print(f"Attendance already marked for {person_name} within the last {ATTENDANCE_INTERVAL_HOURS} hour(s).")
                conn.close()
                return False


        cursor.execute(
            f"INSERT INTO {table_name} (name, date, time) VALUES (%s, %s, %s)",
            (person_name, log_date, log_time)
        )
        conn.commit()
        print(f"Attendance marked for {person_name} in Section {section.upper()}.")

        conn.close()
        return True

    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")
        return False



def get_today_attendance_count(person_name, section):
    """ Count how many distinct days attendance was marked in a section. """
    try:
        conn = mysql.connector.connect(host=DB_HOST, user=DB_USER, passwd=DB_PASSWORD, database=DB_NAME)
        cursor = conn.cursor()
        table_name = f"attendance_{section.lower()}"
        today = datetime.now().date()
        cursor.execute(f"SELECT COUNT(*) FROM {table_name} WHERE name=%s AND date = %s", (person_name, today))
        lecture_count = cursor.fetchone()[0]
        conn.close()
        return lecture_count
    except mysql.connector.Error as err:
        print(f"MySQL Error: {err}")
        return 0


def register_new_user(cap):
    """ Guides the user through the registration process. """
    user_name = input("Enter your name: ").strip()
    if not user_name:
        print("Name cannot be empty. Returning to main menu.")
        return

    while True:
        section = input("Enter your section (a, b, or c): ").strip().lower()
        if section in ['a', 'b', 'c']:
            break
        print(" Invalid section. Please enter 'a', 'b', or 'c'.")

    face_database = load_face_database(section)
    
    # Check for existing user
    for entry in face_database:
        if entry['name'].lower() == user_name.lower():
            print(f"User '{user_name}' already exists in Section {section}. Please choose a different name or proceed to attendance.")
            return
            
    print(f"Starting image capture for '{user_name}' from Section {section}. Please look at the camera and slightly vary your pose and expression.")
    print(f"We will capture {NEW_USER_IMAGES_COUNT} images.")
    
    new_embeddings = []
    captured_count = 0
    start_time = time.time()
    
    while captured_count < NEW_USER_IMAGES_COUNT:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break

        # Display prompt on the frame
        cv2.putText(frame, f"Capturing image {captured_count + 1}/{NEW_USER_IMAGES_COUNT}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, "Please face the camera and blink.", (50, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.imshow("Registration", frame)
        cv2.waitKey(1)

        if time.time() - start_time >= IMAGE_CAPTURE_DELAY:
            try:
                detected_faces = DeepFace.extract_faces(
                    img_path=frame,
                    detector_backend="retinaface",
                    enforce_detection=True,
                    align=True
                )
                if detected_faces:
                    aligned_face = detected_faces[0]['face']
                    embedding_objs = DeepFace.represent(
                        img_path=aligned_face,
                        model_name="ArcFace",
                        detector_backend="skip"
                    )

                    if embedding_objs:
                        embedding = np.array(embedding_objs[0]['embedding'], dtype=np.float32)
                        embedding = l2_normalize(embedding)   # normalize once
                        new_embeddings.append(embedding)
                        captured_count += 1
                        print(f"Captured image {captured_count}/{NEW_USER_IMAGES_COUNT}")
                    else:
                        print(" Could not generate embedding for the face.")
                else:
                    print("No face detected. Please make sure your face is visible.")
            except Exception as e:
                print(f"Error during image capture: {e}")
            
            start_time = time.time()

    cv2.destroyAllWindows()
    
    if new_embeddings:
        new_entry = {
            "name": user_name,
            "embeddings": new_embeddings
        }
        face_database.append(new_entry)
        
        # Save the updated database
        save_face_database(face_database, section)
        
    else:
        print(" Registration failed. No images were captured.")


def l2_normalize(x):
    x = np.array(x, dtype=np.float32)
    return x / (np.linalg.norm(x) + 1e-10)

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-10)

# ---------------- MAIN ----------------
if __name__ == "__main__":
    setup_mysql_database()
    sheet = setup_google_sheets()

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    frame_count = 0
    process_every_n_frames = 3
    prev_time = time.time()
    attendance_marked = set()
    detection_active = False
    
    MAX_PEOPLE = -1
    mode_text = "Mode: Detection OFF"

    LEFT_EYE = [362, 385, 387, 263, 373, 380]
    RIGHT_EYE = [33, 160, 158, 133, 145, 153]

    with mp_face_mesh.FaceMesh(
        max_num_faces=MAX_PEOPLE if MAX_PEOPLE > 0 else 10,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:

        while True:
            # Main menu screen
            if not detection_active:
                ret, frame = cap.read()
                if not ret: break

                menu_text = [
                    "Welcome! Please choose an option:",
                    "Press 'a' to start attendance for Section-A",
                    "Press 'b' to start attendance for Section-B",
                    "Press 'c' to start attendance for Section-C",
                    "Press 'n' to register as a new user",
                    "Press 'q' to quit"
                ]
                y_offset = 70
                for line in menu_text:
                    cv2.putText(frame, line, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                    y_offset += 30

                cv2.imshow("Main Menu", frame)
                
                key = cv2.waitKey(1) & 0xFF
                if key in [ord('a'), ord('b'), ord('c')]:
                    selected_section = chr(key)
                    face_database = load_face_database(selected_section)
                    detection_active = True
                    MAX_PEOPLE = 1
                    mode_text = f"Mode: Detecting 1 person in Section {selected_section.upper()}"
                    print(f" Starting attendance mode for Section {selected_section.upper()}...")
                    cv2.destroyAllWindows()
                elif key == ord('n'):
                    cv2.destroyAllWindows()
                    register_new_user(cap)
                elif key == ord('q'):
                    break
                continue

            # Attendance mode (existing logic)
            ret, frame = cap.read()
            if not ret:
                break
            
            curr_time = time.time()
            fps = 1.0 / (curr_time - prev_time + 1e-10)
            prev_time = curr_time
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results_mesh = face_mesh.process(frame_rgb)
            
            
            if frame_count % process_every_n_frames == 0:
                try:
                    detected_faces_deepface = DeepFace.extract_faces(
                        img_path=frame,
                        detector_backend="retinaface",
                        enforce_detection=False,
                        align=True
                    )
                    
                    if MAX_PEOPLE > 0:
                        detected_faces_deepface = detected_faces_deepface[:MAX_PEOPLE]
                    
                    last_known_faces_data.clear()
                    
                    for detected_face in detected_faces_deepface:
                        aligned_face = detected_face['face']
                        embedding_objs = DeepFace.represent(
                            img_path=aligned_face,
                            model_name="ArcFace",
                            detector_backend="skip",
                            enforce_detection=False
                        )
                        if not embedding_objs:
                            continue
                        
                        current_embedding = l2_normalize(np.array(embedding_objs[0]['embedding'])).astype('float32')
                        best_similarity = -1
                        best_match_name = "Unknown"
                        
                        # Use FAISS for fast search
                        if index is not None:
                            distances, indices = index.search(np.array([current_embedding]), k=1)
                            best_similarity = float(distances[0][0])  # already cosine similarity
                            best_match_name = id_to_name[indices[0][0]]
                        else:
                            best_similarity = -1
                            best_match_name = "Unknown"
                            for entry in face_database:
                                for db_embedding in entry['embeddings']:
                                    sim = cosine_similarity(current_embedding, db_embedding)
                                    if sim > best_similarity:
                                        best_similarity = sim
                                        best_match_name = entry['name']

                        # Apply threshold
                        if best_similarity < SIMILARITY_THRESHOLD:
                            best_match_name = "Unknown"
                            
                        facial_area = detected_face["facial_area"]
                        x, y, w, h = facial_area["x"], facial_area["y"], facial_area["w"], facial_area["h"]
                        x2, y2 = x + w, y + h
                        
                        last_known_faces_data[best_match_name] = {
                            'coords': (x, y, x2, y2),
                            'label': best_match_name,
                            'similarity': best_similarity
                        }
                except Exception as e:
                    print(f"Detection/Embedding error: {e}")
            
            if results_mesh.multi_face_landmarks:
                for face_landmarks in results_mesh.multi_face_landmarks:
                    lm_bbox = cv2.boundingRect(np.array([(int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])) for lm in face_landmarks.landmark]))
                    lm_x, lm_y, lm_w, lm_h = lm_bbox
                    
                    identified_person = None
                    min_dist_sq = float('inf')
                    for name, data in last_known_faces_data.items():
                        deepface_coords = data['coords']
                        df_x, df_y, df_x2, df_y2 = deepface_coords
                        df_w = df_x2 - df_x
                        df_h = df_y2 - df_y
                        
                        # Use center distance to match
                        center_dist_sq = ( (lm_x + lm_w/2) - (df_x + df_w/2) )**2 + ( (lm_y + lm_h/2) - (df_y + df_h/2) )**2
                        if center_dist_sq < min_dist_sq:
                            min_dist_sq = center_dist_sq
                            identified_person = name
                    
                    if identified_person and identified_person in last_known_faces_data:
                        # Eye Blink Liveness
                        left_eye_points = [face_landmarks.landmark[i] for i in LEFT_EYE]
                        right_eye_points = [face_landmarks.landmark[i] for i in RIGHT_EYE]
                        
                        left_ear = eye_aspect_ratio(left_eye_points)
                        right_ear = eye_aspect_ratio(right_eye_points)
                        avg_ear = (left_ear + right_ear) / 2.0
                        
                        liveness_data.setdefault(identified_person, {'blinks': 0, 'consecutive': 0, 'static_frames': 0})
                        
                        if avg_ear < BLINK_THRESHOLD:
                            liveness_data[identified_person]['consecutive'] += 1
                        else:
                            if liveness_data[identified_person]['consecutive'] >= CONSECUTIVE_FRAMES:
                                liveness_data[identified_person]['blinks'] += 1
                                print(f"Liveness Check: Blink detected for {identified_person}. Total blinks: {liveness_data[identified_person]['blinks']}")
                            liveness_data[identified_person]['consecutive'] = 0

                        # Micro-Movement Liveness
                        movement_score = calculate_micro_movement(face_landmarks.landmark, identified_person, frame.shape[1], frame.shape[0])
                        
                        if movement_score < MICRO_MOVEMENT_THRESHOLD:
                            liveness_data[identified_person]['static_frames'] += 1
                            liveness_data[identified_person]['is_moving'] = False
                        else:
                            liveness_data[identified_person]['static_frames'] = 0
                            liveness_data[identified_person]['is_moving'] = True
            
            
            for name, data in last_known_faces_data.items():
                (x1, y1, x2, y2) = data['coords']
                
                face_img = frame[y1:y2, x1:x2]
                
                if face_img.shape[0] > 0 and face_img.shape[1] > 0:
                    mask_label, mask_color = detect_mask(face_img)
                else:
                    mask_label, mask_color = "N/A", (128, 128, 128) # Gray
                if mask_label == "Mask":
                    cv2.put_Text(frame, "Mask Detected - Plese remove it first!", (x1,y2+20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,0,255), 2)
                    if name in liveness_data:
                        liveness_data[name]['blinks'] = 0
                        liveness_data[name]['static_frames'] = 0
                    continue
                
                label = data['label']
                sim = data['similarity']
                
                
                liveness_status = "N/A"
                if name in liveness_data:
                    blinks = liveness_data[name]['blinks']
                    static_frames = liveness_data[name].get('static_frames', 0)
                    
                    
                    if static_frames >= CONSECUTIVE_STATIC_FRAMES:
                        detection_active = False
                        mode_text = "MODE: Detection OFF (STATIC FACE DETECTED)"
                        print("Static face detected! Returning to main menu.")
                        cv2.putText(frame, "STATIC FACE DETECTED! Returning to menu.", (x1, y2 + 20),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                        liveness_data.clear()
                        previous_landmarks.clear() 
                        break 
                    
                    is_live_blink = blinks >= LIVENESS_BLINKS_REQUIRED
                    is_live_movement = static_frames < CONSECUTIVE_STATIC_FRAMES
                    
                    if is_live_blink and is_live_movement:
                        liveness_status = "Live"
                        if name != "Unknown":
                            success = log_attendance_to_mysql(name, selected_section)
                            if success and sheet:
                                log_attendance_to_gsheet(sheet, name, selected_section)
                                lecture_count = get_today_attendance_count(name, selected_section)
                                print(f" {name} has {lecture_count} total days of attendance marked in Section {selected_section.upper()}.")
                            
                            if name in liveness_data:
                                liveness_data[name]['blinks'] = 0
                                liveness_data[name]['static_frames'] = 0
                    elif not is_live_movement:
                        liveness_status = f"Static ({static_frames}/{CONSECUTIVE_STATIC_FRAMES})"
                    else:
                        liveness_status = f"Blinks: {blinks}/{LIVENESS_BLINKS_REQUIRED}"
                
                # Draw results
                cv2.rectangle(frame, (x1, y1), (x2, y2), mask_color, 2)
                cv2.putText(frame, f"{label} ({sim:.2f})", (x1, y1 - 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv2.putText(frame, f"Mask: {mask_label}", (x1, y1 - 45),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, mask_color, 2)
                cv2.putText(frame, f"Liveness: {liveness_status}", (x1, y1 - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                if name != "Unknown":
                    lecture_count = get_today_attendance_count(name, selected_section)  # pass section
                    cv2.putText(frame, f"Lecture Today: {lecture_count}", (x1, y1 - 15),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

            cv2.putText(frame, mode_text, (30, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, "Press 'p' to pause, 'q' to quit.", (30, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            cv2.imshow("Face Recognition, Liveness & Mask Detection", frame)
            
            frame_count += 1
            key = cv2.waitKey(1) & 0xFF

            if key == ord('q'):
                break
            
            elif key == ord('p'):
                detection_active = False
                mode_text = "Mode: Detection PAUSED"
                print("⏸ Detection paused. Press 'a' to resume.")

    cap.release()
    cv2.destroyAllWindows()


Mask detection model loaded successfully.
MySQL Database 'attendance_db' setup successful.
 Connected to Google Sheets (Sections A, B, C).
Face database for Section a loaded successfully.
FAISS index built with 160 embeddings for Section a.
 Starting attendance mode for Section A...
Liveness Check: Blink detected for Alok. Total blinks: 1
Liveness Check: Blink detected for Unknown. Total blinks: 1
Liveness Check: Blink detected for Unknown. Total blinks: 2
Liveness Check: Blink detected for Alok. Total blinks: 2
Attendance marked for Alok in Section A.
Attendance logged for Alok in Section A (Google Sheets).
 Alok has 1 total days of attendance marked in Section A.
Liveness Check: Blink detected for Alok. Total blinks: 1
Liveness Check: Blink detected for Alok. Total blinks: 2
Attendance already marked for Alok within the last 1 hour(s).
